In [5]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

import pandas as pd
import csv
import numpy as np
import datetime
import re
import warnings
from IPython.display import HTML, display
import requests
import sys
from scipy.optimize import curve_fit

warnings.filterwarnings('ignore')
plt.ioff()

%config InlineBackend.figure_format = 'svg'

In [6]:

def logistic(x, a, b, c, d):
    return a / (d + np.exp(-c * x + b))

def obs_pred_rsquare(obs, pred):
    return 1 - sum((obs - pred) ** 2) / sum((obs - np.mean(obs)) ** 2)

def exponential(x, a, b, c, d):
    return a * np.exp(b * x) + c



def get_logistic(obs_x, obs_y):

    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    try:
        popt, pcov = curve_fit(logistic, obs_x, obs_y)
        pred_y = logistic(obs_x, *popt)
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = logistic(forecasted_x, *popt)
        
    except:
        print('Logistic failed to fit. Using 3rd degree polynomial.')
        z = np.polyfit(obs_x, obs_y, 3)
        p = np.poly1d(z)
        pred_y = p(obs_x)
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = p(forecasted_x)
        
    return forecasted_y, forecasted_x, pred_y



def get_exponential(obs_x, obs_y):
    
    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    try:
        popt, pcov = curve_fit(exponential, obs_x, obs_y)
        pred_y = exponential(obs_x, *popt)
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = logistic(forecasted_x, *popt)
        
    except:
        print('Exponential failed to fit. Using 3rd degree polynomial.')
        z = np.polyfit(obs_x, obs_y, 3)
        p = np.poly1d(z)
        pred_y = p(obs_x)
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = p(forecasted_x)
        
    return forecasted_y, forecasted_x, pred_y
        


def get_polynomial(obs_x, obs_y):
    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    try:
        z = np.polyfit(obs_x, obs_y, 3)
        p = np.poly1d(z)
        pred_y = p(obs_x)
            
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = p(forecasted_x)
    except:
        pass
    
    return forecasted_y, forecasted_x, pred_y



def get_best(obs_x, obs_y, model, df_sub):
    #global df_main
    
    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    best_r2 = -(10**30)
    best_model = str()
    best_loc = str()
    best_pred_y = []
    best_forecasted_x = []
    best_forecasted_y = []
    
    bench_x, bench_y = [], []
    
    locs = list(set(df_sub['Province/State']))
    for loc in locs:
        df_benchmark = df_sub[df_sub['Province/State'] == loc]
        bench = df_benchmark.iloc[0,6:].values
        
        bench_y = []
        for i, val in enumerate(bench):
            if len(bench_y) > 0 or val > 0:
                bench_y.append(val)

        bench_x = list(range(len(bench_y)))
        if len(bench_x) <= len(obs_x) or sum(bench_y) <= sum(obs_y): 
            continue
            
        z = np.polyfit(bench_x, bench_y, 10)
        p = np.poly1d(z)
        pred_y = p(obs_x)
            
        forecasted_x = np.array(list(range(max(obs_x) + 14)))
        forecasted_y = p(forecasted_x)
        
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
        
        if obs_pred_r2 > best_r2:
            best_r2 = float(obs_pred_r2)
            best_loc = loc
            best_pred_y = pred_y
            best_forecasted_x = forecasted_x
            best_forecasted_y = forecasted_y

    return best_r2, best_loc, best_pred_y, best_forecasted_x, best_forecasted_y



def fit_curve(obs_x, obs_y, model, df_sub):

    obs_x = list(range(len(obs_y)))
    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    best_loc = str()
    
    if model == 'Best Benchmark':
        obs_pred_r2, best_loc, pred_y, forecasted_x, forecasted_y = get_best(obs_x, obs_y, model, df_sub)
    elif model == 'logistic':
        forecasted_y, forecasted_x, pred_y = get_logistic(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
    elif model == 'exponential':
        forecasted_y, forecasted_x, pred_y = get_exponential(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
    elif model == 'polynomial':
        forecasted_y, forecasted_x, pred_y = get_polynomial(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
        
    return obs_pred_r2, model, best_loc, obs_x, pred_y, forecasted_x, forecasted_y


In [7]:
class App_GetFits:
    
    def __init__(self, df):
        
        # model: 'logistic'; 'exponential'; 'polynomial';
        # query: Any location available within the dataframe
        # refer: Any location available within the dataframe
        
        self._df = df
        available_indicators2 = list(set(self._df['Province/State']))
        available_indicators2.sort()
        
        self._1_dropdown = self._create_dropdown(['logistic', 'exponential', 'polynomial', 'Best Benchmark'], 0, label = 'Choose model:')
        self._2_dropdown = self._create_dropdown(available_indicators2, 92, label = 'Choose location:')
        self._plot_container = widgets.Output()
        
        _app_container = widgets.VBox(
            [widgets.VBox([self._1_dropdown, self._2_dropdown]),
            self._plot_container], layout=widgets.Layout(align_items='center', flex='0 0 auto'))
                
        # 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset'
        self.container = widgets.VBox([
            widgets.HBox([
                _app_container
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
        
    @classmethod
    def from_url(cls):
        
        _URL = 'https://raw.githubusercontent.com/klocey/ResourceDemand/master/notebooks/COVID-CASES-DF.txt'
        df = pd.read_csv(_URL, sep='\t')        
        
        return cls(df)
        
        
    def _create_dropdown(self, indicators, initial_index, label):
        dropdown = widgets.Dropdown(options=indicators, 
                                    layout={'width': 'initial'},
                                    style={'description_width': 'initial'},
                                    value=indicators[initial_index],
                                   description=label)
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    
    def _get_fit(self, model, loc):
        # model = 'logistic', 'exponential', 'polynomial', 'Best Benchmark'
        
        df_sub = self._df[self._df['type'] == 'Confirmed']
        
        yi = list(df_sub)
        dates = yi[6:]
        
        df_loc = df_sub[df_sub['Province/State'] == loc]
        focal = df_loc.iloc[0,6:].values
        
        y = []
        for i, val in enumerate(focal):
            if len(y) > 0 or val > 0:
                y.append(val)

        x = list(range(len(y)))
        obs_pred_r2, model, loc, obs_x, pred_y, forecasted_x, forecasted_y = fit_curve(x, y, model, df_sub)
        
        
        
        fig = plt.figure()
        fig.add_subplot(111)
        ax = plt.gca()
        lab1 = r'$r^{2}$' + ' = ' + str(np.round(obs_pred_r2, 3))
        
        y = np.array(y)
        y[y < 0] = 0
        plt.plot(obs_x, y, c='0.2', linewidth=3, label='Observed')
        
        forecasted_y = np.array(forecasted_y)
        forecasted_y[forecasted_y < 0] = 0
        forecast_vals = forecasted_y[-14:]
        d7 = forecast_vals[6]
        d14 = forecast_vals[-1]
        plt.plot(forecasted_x, forecasted_y, c='Crimson', linewidth=3, label='Forecasted:\n'+str(int(d7))+' in 7 days\n'+str(int(d14))+' in 14 days')
        
        pred_y = np.array(pred_y)
        pred_y[pred_y < 0] = 0
        
        plt.plot(obs_x, pred_y, c='steelblue', linewidth=3, label='Predicted, ' + lab1)
        
        leg = ax.legend(handlelength=0, handletextpad=0, fancybox=False,
                        loc='best', frameon=False, fontsize=14)
        
        for line,text in zip(leg.get_lines(), leg.get_texts()):
            text.set_color(line.get_color())

        for item in leg.legendHandles: 
            item.set_visible(False)

        plt.xlabel('Date', fontsize=14)
        plt.ylabel('Confirmed cases', fontsize=14)
        
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        
        model = self._1_dropdown.value
        focal_loc = self._2_dropdown.value
        
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._get_fit(model, focal_loc)
            plt.show()
    

In [8]:
%config InlineBackend.figure_format = 'svg'

app1 = App_GetFits.from_url()
app2 = App_GetFits.from_url()
app3 = App_GetFits.from_url()

# Put the views into a layout grid
grid = widgets.GridspecLayout(1, 3, layout=widgets.Layout(justify_content='center'))
grid[0, 0] = app1.container
#grid[0, 1] = app2.container
#grid[0, 2] = app3.container

app_contents = [grid]
app = widgets.VBox(app_contents)
    
display(app)